In [16]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import pickle as pkl
from torchsummary import summary


In [9]:
x = pkl.load(open('x.pkl','rb'))
y = pkl.load(open('y.pkl','rb'))
x = x/255

In [11]:
x.shape

(456, 100, 100)

In [21]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv = nn.Conv2d(1, 28, kernel_size=5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(28, 56, kernel_size=5)
        self.pool2 = nn.MaxPool2d(2)
        self.hidden= nn.Linear(56*22*22, 300)
        self.drop = nn.Dropout(0.2)
        self.hidden2 = nn.Linear(300, 100)
        self.out = nn.Linear(100, 6)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.conv(x)) # [batch_size, 28, 96, 96]
        x = self.pool(x) # [batch_size, 28, 48, 48]
        x = self.act(self.conv2(x)) # [batch_size, 56, 44, 44]
        x = self.pool(x)# [batch_size, 56, 22, 22]
        x = x.view(x.size(0), -1) # [batch_size, 28*13*13=4732]
        x = self.act(self.hidden(x))# [batch_size, 128]
        x = self.act(self.hidden2(x)) 
        x = self.drop(x)
        x = self.out(x) # [batch_size, 10]
        return x


In [22]:
model = NeuralNet()

In [23]:
summary(model, (1, 100, 100))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 28, 96, 96]             728
              ReLU-2           [-1, 28, 96, 96]               0
         MaxPool2d-3           [-1, 28, 48, 48]               0
            Conv2d-4           [-1, 56, 44, 44]          39,256
              ReLU-5           [-1, 56, 44, 44]               0
         MaxPool2d-6           [-1, 56, 22, 22]               0
            Linear-7                  [-1, 300]       8,131,500
              ReLU-8                  [-1, 300]               0
            Linear-9                  [-1, 100]          30,100
             ReLU-10                  [-1, 100]               0
          Dropout-11                  [-1, 100]               0
           Linear-12                    [-1, 6]             606
Total params: 8,202,190
Trainable params: 8,202,190
Non-trainable params: 0
---------------------------